In [ ]:
import os
import pandas as pd
import re

: 

**Đọc File**

In [7]:
working_dir = os.path.dirname(os.path.abspath(''))
folder_data_path = f"{working_dir}\\data\\raw"
raw_data = folder_data_path + "\\Final_TopCV.csv"
df = pd.read_csv(raw_data).drop_duplicates(subset=['job_id'])
df.dropna(subset = ['job_decription'], inplace=True)

In [8]:
print('Dữ liệu có: ',len(df.columns),' cột và ',len(df), ' dòng trước khi tiền xử lý')
print('Các cột bao gồm: ', list(df.columns))

Dữ liệu có:  8  cột và  3146  dòng trước khi tiền xử lý
Các cột bao gồm:  ['job_id', 'job_title', 'job_salary', 'job_decription', 'job_tags', 'company_address', 'company_name', 'company_size']


In [22]:
df[len(df)-1:len(df)].iloc[0]

job_id                                                       1114545
job_title          Chuyên Viên Phân Tích Nghiệp Vụ - Business Ana...
job_salary                                          \nTới 15 triệu\n
job_decription     Mô tả công việc<br><br><div class="job-descrip...
job_tags                           Business Analyst, Data Analysis, 
company_address                                               Hà Nội
company_name                                              Epic Group
company_size                                       100-499 nhân viên
Name: 3190, dtype: object

**Tiền xử lý**

Xử lý cho biến **job_salary**

In [4]:
min_salary = []
max_salary = []

# Tỉ suất chuyển đổi VND với USD theo trang https://wise.com/vn/currency-converter/usd-to-vnd-rate?amount=1
exchange_rate = 25318

# Đơn vị tiền tệ trong dataset
currency = ['triệu', 'USD']

# Giới hạn dưới và giới hạn trên của job_salary
threshold = ['Tới', 'Trên']

for row in df['job_salary']:
    row = row.replace("\n","")
    row_split = row.split(' ')
    min = ''
    max = ''
    # Nếu choices là 0 thì min cho số tiếp theo, 1 thì max cho số tiếp theo
    choices = 0
    for data in row_split:
        # Dữ liệu sẽ chia ra làm 3 phần: số, đơn vị tiền tệ và giới hạn trên, dưới nếu có
        if data in threshold:
            if data == threshold[0]:
                choices = 1
        elif data in currency:
            if data == currency[1]:
                if min != '':
                    min*=exchange_rate
                if max != '':
                    max*=exchange_rate
        elif data != '-':
            if choices == 0:
                min = data
                choices = 1
            else:
                max = data
    min_salary.append(min)
    max_salary.append(max)

df = df.assign(min_salary = min_salary, max_salary = max_salary)
df

,job_id,job_title,job_salary,job_decription,job_tags,company_address,company_name,company_size,min_salary,max_salary
0,1394422,Chuyên Viên Quản Trị Vận Hành Hệ Thống Hạ Tầng...,\n,"Mô tả công việc<br><br><div class=""job-descrip...",NaN,Hà Nội,Ngân hàng Hợp tác xã Việt Nam,1000+ nhân viên,,
1,1400446,Web Developer,\n,"Mô tả công việc<br><br><div class=""job-descrip...",NaN,Hồ Chí Minh,CÔNG TY TNHH THƯ VIỆN PHÁP LUẬT,100-499 nhân viên,,
2,1398031,Lập Trình Viên Java (Springboot) - Thu Nhập Up...,\n25 - 40 triệu\n,"Mô tả công việc<br><br><div class=""job-descrip...","Java, Spring Boot,",Hà Nội,Công ty cổ phần Công nghệ thông tin Phú Minh,25-99 nhân viên,25,40
3,1401180,Chuyên Viên Phát Triển Ứng Dụng CNNH (Java Dev),\n,"Mô tả công việc<br><br><div class=""job-descrip...",NaN,Hà Nội,Ngân Hàng TMCP An Bình,5000+ nhân viên,,
4,1399001,Business Analyst ( BA ) - Phân Tích Nghiệp Vụ,\n10 - 20 triệu\n,"Mô tả công việc<br><br><div class=""job-descrip...",NaN,Hà Nội,CÔNG TY CỔ PHẦN CÔNG NGHỆ SOPEN VIỆT NAM,25-99 nhân viên,10,20
...,...,...,...,...,...,...,...,...,...,...
3186,1379426,Lập Trình Viên IT Phần Mềm,\n10 - 15 triệu\n,"Mô tả công việc<br><br><div class=""job-descrip...","Công Nghệ Thông Tin,",Bắc Giang,CÔNG TY TNHH KHOA HỌC KỸ THUẬT NĂNG LƯỢNG MẶT ...,500-1000 nhân viên,10,15
3187,1066695,IOS Developer,\n,"Mô tả công việc<br><br><div class=""job-descrip...","iOS, Objective-C, Swift,",Hà Nội,CÔNG TY CỔ PHẦN AHT TECH,100-499 nhân viên,,
3188,1177885,Trưởng Nhóm Quản Lý Dự Án – Tiếng Nhật (N1/N2/N3),\nTới 48 triệu\n,"Mô tả công việc<br><br><div class=""job-descrip...",NaN,Hà Nội & 2 nơi khác,Công ty TNHH NTT DATA VIỆT NAM,100-499 nhân viên,,48
3189,1384856,Global Business Development,\n,"Mô tả công việc<br><br><div class=""job-descrip...",NaN,Hồ Chí Minh,CÔNG TY TNHH GIANTY VIỆT NAM,100-499 nhân viên,,


Xử lý cho biến **job_decription**

In [5]:
job_description = []
job_request = []
job_benefit = []

for row in df['job_decription']:
    row = row.replace("</div><br>", "<br><br>")
    row_split = row.split("<br><br>")
    
    # Regex để xóa tag html
    p = re.compile(r'<.*?>')
    
    # Đưa vào mảng
    job_description.append(p.sub('',row_split[1]))
    job_request.append(p.sub('',row_split[3]))
    job_benefit.append(p.sub('',row_split[5]))

df = df.assign(descriptions = job_description, benefits = job_benefit, requests = job_request)

Xóa những biến không cần dùng và xuất 1 file mới

In [6]:
df = df.drop(columns=['job_decription', 'job_salary'])
df.to_csv(folder_data_path + "\\Final_TopCV_new.csv")